<a href="https://colab.research.google.com/github/Andrea91s/Andrea91s/blob/main/Immobiliare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import csv
import re
import itertools
import warnings
warnings.filterwarnings("ignore")

In [2]:
#connect to the website and fetch information
def connect(web_addr):
    resp = requests.get(web_addr)
    return BeautifulSoup(resp.content, "html.parser")

In [3]:
# scraping raw html to find districts in the city
def get_areas(website):
    soup = connect(website)
    areas=[]
    for link_quart in soup.find_all('a', class_='in-breadcrumb__dropdownLink'):
        for i in range(len(link_quart.text.split(','))):
          areas.append(link_quart.text.split(',')[i])
    areas = [stringa.strip() for stringa in areas]
    urls = []

    for area in areas:
        url = website + area.replace(' ','-').lower()
        urls.append(url)

    return areas, urls

In [4]:
#filtering districts to remove the unwanted ones (this should become a parameter)
def filtering_districts(filtered_areas, filtered_districts):
  lower_filtered =[]
  for word in filtered_areas:
      lower_filtered.append("-".join(word.lower().split()))

  filtered_url=[]
  for element in url:
    for word in lower_filtered:
      if word in element:
        filtered_url.append(element)
  return filtered_url

In [5]:
def get_pages(main):
    try:
        soup = connect(main)
        n_pages = [_.get_text(strip=True) for _ in soup.find('ul', {'class': 'pagination pagination__number'}).find_all('li')]
        last_page = int(n_pages[-1])
        pages = [main]

        for n in range(2,last_page+1):
            page_num = "/?pag={}".format(n)
            pages.append(main + page_num)
    except:
        pages = [main]

    return pages

In [6]:
website = "https://www.immobiliare.it/vendita-case/torino/"
areas,url=get_areas(website)

filtered_areas=['Cit Turin', 'Centro', 'Santa Rita', 'Crocetta', 'San Secondo','Cenisia', 'San Salvario']

filtered_districts=[elemento for elemento in areas if elemento in filtered_areas]
filtered_url=filtering_districts(filtered_areas, filtered_districts)
print("Those are the selected districts \n")

print(filtered_url)


Those are the selected districts 

['https://www.immobiliare.it/vendita-case/torino/cit-turin', 'https://www.immobiliare.it/vendita-case/torino/santa-rita', 'https://www.immobiliare.it/vendita-case/torino/crocetta', 'https://www.immobiliare.it/vendita-case/torino/san-secondo', 'https://www.immobiliare.it/vendita-case/torino/cenisia', 'https://www.immobiliare.it/vendita-case/torino/san-salvario', 'https://www.immobiliare.it/vendita-case/torino/centro']


In [13]:
announces_links = []
for url_page in tqdm(filtered_url):
   soup = connect(url_page)
   href=soup.find_all('a', class_='in-pagination__item')
   max_pages=(int(href[-1]['href'].split('pag=')[1]))
   for page in range(max_pages):
    new_url_page=url_page + "/?pag=" + str(page+1)
    print('Scraping',new_url_page)
    soup_page=connect(new_url_page)
    all_apartments=[a['href'] for a in soup_page.find_all('a', class_='in-card__title')]
    announces_links.append(all_apartments)
announces_links=list(itertools.chain.from_iterable(announces_links))



  0%|          | 0/7 [00:00<?, ?it/s]

Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=1
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=2
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=3
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=4
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=5
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=6
Scraping https://www.immobiliare.it/vendita-case/torino/cit-turin/?pag=7
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=1
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=2
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=3
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=4
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=5
Scraping https://www.immobiliare.it/vendita-case/torino/santa-rita/?pag=6
Scraping https://www.immobiliare.it/vendita-c

In [14]:
len(announces_links)

1670

In [60]:
df_scrape=pd.DataFrame(index=range(len(announces_links)))
df_scrape

""
0
1
2
3
4
...
1665
1666
1667
1668


In [61]:
df_scrape=pd.DataFrame()
for url_page in tqdm(announces_links):
    data = connect(url_page)
    info = data.find_all('dl', class_='in-realEstateFeatures__list')
    comp_info = pd.DataFrame()
    cleaned_id_text = []
    cleaned_id__attrb_text = []
    for n in range(len(info)):
        for i in info[n].find_all('dt'):
            cleaned_id_text.append(i.text)
        for i in info[n].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)

    comp_info['Id'] = cleaned_id_text
    comp_info['Attribute'] = cleaned_id__attrb_text
    temp_df = pd.DataFrame(columns=comp_info['Id'].values)
    temp_df.loc[len(temp_df), :] = comp_info['Attribute'].values[0:len(comp_info['Id'].values)]
    temp_df['url']=url_page
    df_scrape = df_scrape.append(temp_df, sort=False)
    df_scrape
df_scrape.to_csv('immobiliare_df.csv')

  0%|          | 0/1670 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [62]:
df_scrape

,Riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,altre caratteristiche,prezzo,spese condominio,cauzione,stato,riscaldamento,Efficienza energetica,url
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
temp_df

,Riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,altre caratteristiche,prezzo,spese condominio,cauzione,stato,riscaldamento,Efficienza energetica,url
0,Vittorio210162mq - 09/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,162 m² - Vedi dettaglio,"4 (2 camere da letto, 2 altri), 2 bagni, cucin...","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 318.000,€ 130/mese,Non indicata,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",E189 kWh/m² anno,https://www.immobiliare.it/annunci/105273157/


In [55]:
df_scrape.append(temp_df)

,Riferimento e Data annuncio,contratto,tipologia,superficie,locali,piano,totale piani edificio,disponibilità,altre caratteristiche,prezzo,spese condominio,cauzione,stato,riscaldamento,Efficienza energetica,url
0,Vittorio117 - 11/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,117 m² - Vedi dettaglio,"3 (2 camere da letto, 1 altro), 1 bagno, cucin...","1°, con ascensore, con accesso disabili",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 230.000,€ 120/mese,Non indicata,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",E189 kWh/m² anno,https://www.immobiliare.it/annunci/105303639/
0,Vittorio45 - 09/08/2023,Vendita,Appartamento | Classe immobile signorile,45 m² - Vedi dettaglio,"1 camera da letto, 1 bagno, cucina abitabile","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaPorta bl...,€ 88.000,€ 40/mese,Non indicata,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a metano",E189 kWh/m² anno,https://www.immobiliare.it/annunci/105273371/
0,Vittorio210162mq - 09/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,162 m² - Vedi dettaglio,"4 (2 camere da letto, 2 altri), 2 bagni, cucin...","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 318.000,€ 130/mese,Non indicata,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",E189 kWh/m² anno,https://www.immobiliare.it/annunci/105273157/
0,Vittorio210162mq - 09/08/2023,Vendita,Appartamento | Intera proprietà | Classe immob...,162 m² - Vedi dettaglio,"4 (2 camere da letto, 2 altri), 2 bagni, cucin...","1°, con ascensore",7 piani,Libero,Cancello elettricoMansardaFibra otticaArmadio ...,€ 318.000,€ 130/mese,Non indicata,Buono / Abitabile,"Centralizzato, a radiatori, alimentato a metano",E189 kWh/m² anno,https://www.immobiliare.it/annunci/105273157/
